In [1]:
import muspy
from harmony_tokenizers import ChordSymbolTokenizer, RootTypeTokenizer
from tqdm import tqdm
import os

/home/maximos/miniconda/envs/midi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
gjt_path = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies/'
gjt_list = os.listdir(gjt_path)
print(len(gjt_list))

650


In [3]:
gjt_pieces = []
for i in tqdm(range(len(gjt_list))):
    g = muspy.read_musicxml(gjt_path + gjt_list[i])
    gjt_pieces.append(g)

100%|██████████| 650/650 [00:04<00:00, 134.95it/s]


In [4]:
print(len(gjt_pieces))

650


In [ ]:
print('ChordSymbolTokenizer')
chordSymbolTokenizer = ChordSymbolTokenizer()
print('len(chordSymbolTokenizer.vocab): ', len(chordSymbolTokenizer.vocab))
toks_cs = chordSymbolTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_cs['tokens'][0]))
print(toks_cs['tokens'][0])
print(toks_cs['ids'][0])

ChordSymbolTokenizer
len(chordSymbolTokenizer.vocab):  370
650
example sentence length:  110
['bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'E:min7', 'bar', 'position_0x0', 'A:7', 'bar', 'position_0x0', 'D:min7', 'bar', 'position_0x0', 'G:7', 'bar', 'position_0x0', 'C:maj7', 'bar', 'position_0x0', 'D:min7', 'position_2x0', 'G:7', 'bar', 'position_0x0', 'G:min7', 'position_0x0', 'C:7', 'position_0x0', 'F:maj', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'G:min7', 'bar', 'position_0x0', 'C:7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'F:maj7', 'bar', 'position_0x0', 'A:maj7', 'position_2x0', 'A#:maj7', 'bar

In [6]:
print('RootTypeTokenizer')
rootTypeTokenizer = RootTypeTokenizer()
print('len(rootTypeTokenizer.vocab): ', len(rootTypeTokenizer.vocab))
toks_rt = rootTypeTokenizer(gjt_pieces)
print('example sentence length: ', len(toks_rt['tokens'][0]))
print(toks_rt['tokens'][0])
print(toks_rt['ids'][0])

RootTypeTokenizer
len(rootTypeTokenizer.vocab):  63
example sentence length:  149
['bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'E', 'min7', 'bar', 'position_0x0', 'A', '7', 'bar', 'position_0x0', 'D', 'min7', 'bar', 'position_0x0', 'G', '7', 'bar', 'position_0x0', 'C', 'maj7', 'bar', 'position_0x0', 'D', 'min7', 'position_2x0', 'G', '7', 'bar', 'position_0x0', 'G', 'min7', 'position_0x0', 'C', '7', 'position_0x0', 'F', 'maj', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'G', 'min7', 'bar', 'position_0x0', 'C', '7', 'bar', 'position_0x0', 'F', 'maj7', 'bar', 'position_0x0', 'F', 'm